Extract Data from jobvision..

In [21]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time
from urllib.parse import urlparse, parse_qs

In [2]:
# service=Service(ChromeDriverManager().install())
driver = webdriver.Chrome()
base_url="https://jobvision.ir/jobs/category/data-science?page="


In [ ]:
jobs_list = []
now = datetime.now()

driver.get(base_url + "1")

pages_btn = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "pagination"))
    )

btn_list = pages_btn.find_elements(By.XPATH, "./*")
last_page = btn_list[-1]

last_page.click()

current_url = driver.current_url
parsed_url = urlparse(current_url)
query_params = parse_qs(parsed_url.query)
pages = int(query_params.get('page')[0])


for page in range(1, pages + 1):
    url = base_url + str(page)
    driver.get(url)

    jobs = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, "job-card-list"))
    )

    jobs_element_list = jobs.find_elements(By.TAG_NAME, "job-card")

    for job in jobs_element_list:

        # Get information about job postings from post

        title = job.find_element(By.CLASS_NAME, "job-card-title")

        parent = title.find_element(By.XPATH, "..")
        childs_list = parent.find_elements(By.XPATH, "./*")

        company = childs_list[1]

        job_info_element = childs_list[2]

        if len(childs_list) == 5:
            facility = childs_list[3].text
        else:
            facility = ""
        shipping_time = childs_list[-1]

        job_info_list = job_info_element.find_elements(By.TAG_NAME, "span")

        if len(job_info_list) == 3:
            region, _, salary = job_info_list
            salary = salary.text
        else:
            region = job_info_list[0]
            salary = "an argumant"

        extractec_num = "".join(filter(lambda x: x.isnumeric(), shipping_time.text))
        if extractec_num != "":
            release_time = now - timedelta(days=int(extractec_num))
        else:
            release_time = now - timedelta(days=0)

        # Go to any of the job position

        title.click()

        get_job_section = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.TAG_NAME, "app-job-specification"))
        )

        job_info_list = get_job_section.find_elements(By.CLASS_NAME, "text-muted")
        working_hours, type_of_cooperation = job_info_list[:2]
        try:
            requirement_list = get_job_section.find_elements(By.TAG_NAME, "app-tag")
            requirement_software_list = [i.text for i in requirement_list]
        except:
            requirement_software_list = []

        jobs_list.append(
            {
                "shippingDate": release_time.date(),
                "title": title.text,
                "company": company.text,
                "salary": salary,
                "province": region.text,
                "workingHours": working_hours.text,
                "typeOfCooperation": type_of_cooperation.text,
                "facility": facility,
                "tagList": requirement_software_list,
            }
        )


df = pd.DataFrame(jobs_list)

8

In [5]:
df_copy = df


df_copy.to_csv('job_data_optimized.csv', index=False)